# test 
Test the connection with a single call


In [2]:
#imports
import requests
import time

boxGTal = '593fc68947df4f001113f9eb'
baseURLOpenSense ='https://api.opensensemap.org/boxes/'
try:
    resp = requests.get(baseURLOpenSense+boxGTal)
    box = resp.json()
except TimeoutError:
    print('there was an timeout error')
#except:
#    print("Unexpected error:")

In [3]:
box["name"]


'Freiburg im Breisgau'

# read every interval

In [4]:
#%pip install schedule

In [5]:
from confluent_kafka import Producer

In [6]:
p = Producer({'bootstrap.servers': 'kafka-1:19092,kafka-2:19093'})
boxGTal = '593fc68947df4f001113f9eb'
baseURLOpenSense ='https://api.opensensemap.org/boxes/'
topicNameRaw="raw"
topicNameCleanTemperature="clean_latest"

In [7]:
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))


In [8]:
import schedule
import time
import requests
import json

lastone = ""
def printTemperatureFromBox(s):
    print('{} temperature: {} {}'
              .format(s['lastMeasurement']['createdAt'],
                         s['lastMeasurement']['value'],
                         s['unit']))
        
def job():
    #print("start reading")
    resp = requests.get(baseURLOpenSense+boxGTal)
    #print("finished reading")
    box = resp.json()
    boxData=json.dumps(box)
    p.produce("raw", key=boxGTal, value = boxData.encode('utf-8'), callback=delivery_report)


# schedule the job every minute  
schedule.every(10).seconds.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 